In [13]:
from typing import Dict, List
from datamodel import Order, OrderDepth, TradingState, Trade
import json
import math
import statistics
from datamodel import OrderDepth, TradingState, Order, ConversionObservation
from typing import List

class Trader:
    def __init__(self):
        # Learned model parameters
        self.intercept = 275.811207
        self.coef_sugar = 2.771193
        self.coef_sunlight = -3.127103
        # Threshold (buffer) around fair price to avoid overtrading
        self.margin = 1.0  # 1 seashell

    def run(self, state: TradingState):
        orders = {}
        # --- Handle MACARONS trading using regression model ---
        product = "MAGNIFICENT_MACARONS"
        if product in state.order_depths:
            od: OrderDepth = state.order_depths[product]
            conv_obs: ConversionObservation = state.observations.get(product)

            # Compute fair price
            fair_price = (
                self.intercept +
                self.coef_sugar * conv_obs.sugarPrice +
                self.coef_sunlight * conv_obs.sunlightIndex
            )
            # Include daily storage cost of 0.1 per unit if we hold overnight
            storage_cost = 0.1
            fair_price -= storage_cost

            # Best available prices
            best_ask = min(od.sell_orders) if od.sell_orders else None
            best_bid = max(od.buy_orders)  if od.buy_orders  else None

            mac_orders: List[Order] = []

            # BUY logic: if ask < fair_price - margin, buy up to limit
            if best_ask is not None and best_ask < fair_price - self.margin:
                ask_vol = od.sell_orders[best_ask]
                qty = min(-ask_vol, 10)  # limit order size small
                mac_orders.append(Order(product, best_ask, qty))

            # SELL logic: if bid > fair_price + margin, sell up to limit
            if best_bid is not None and best_bid > fair_price + self.margin:
                bid_vol = od.buy_orders[best_bid]
                qty = min(bid_vol, 10)
                mac_orders.append(Order(product, best_bid, -qty))

            orders[product] = mac_orders

        # --- Conversion for MACARONS if sunlight is strongly below CSI ---
        # If sunlight index remains below critical value for a long period,
        # demand panic should drive price up, so convert short positions to long
        CSI = 65
        if conv_obs and conv_obs.sunlightIndex < CSI:
            # request conversion of up to 10 units to cover a short or build a long
            orders_conversions = min(abs(state.position.get(product, 0)), 10)
        else:
            orders_conversions = 0

        return orders, orders_conversions, ""

ModuleNotFoundError: No module named 'datamodel'